In [43]:
import pandas as pd
import zstandard as zstd
import os
import numpy as np
from collections import Counter

Decompressing all the .zst files with the zstandard library, and a loop repeating the process to all files ending with .csv.zst.

In [3]:
data_dir = "trendingYT"
output_dir = "csv-extraction"
os.makedirs(output_dir, exist_ok=True)
csv_zst_files = [f for f in os.listdir(data_dir) if f.endswith(".csv.zst")]

for file_name in csv_zst_files:
    file_path = os.path.join(data_dir, file_name)
    base_name = os.path.splitext(file_name)[0]
    csv_path = os.path.join(output_dir, base_name)
    with open(file_path, 'rb') as compressed:
        dctx = zstd.ZstdDecompressor()
        with open(csv_path, 'wb') as out_csv:
            out_csv.write(dctx.decompress(compressed.read()))


Creating the Country column for the Canada Dataframe. More info about the encoding choices later

In [4]:
# for files in output_dir:
df_ca = pd.read_csv(output_dir + "/CAvideos.csv", encoding = "utf-8", encoding_errors = "replace")
df_ca["country"] = "CA"
df_ca.head()

# df_us = pd.read_csv(output_dir + "/USvideos.csv")
# df_us["country"] = "US"
# df_us.head()


,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,country
0,n1WpP7iowLc,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,2017-11-10T17:00:03.000Z,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787425,43420,125882,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. Beyoncé i...,CA
1,0dBIkQ4Mz1M,17.14.11,PLUSH - Bad Unboxing Fan Mail,iDubbbzTV,23,2017-11-13T17:00:00.000Z,"plush|""bad unboxing""|""unboxing""|""fan mail""|""id...",1014651,127794,1688,13030,https://i.ytimg.com/vi/0dBIkQ4Mz1M/default.jpg,False,False,False,STill got a lot of packages. Probably will las...,CA
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146035,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...,CA
3,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095828,132239,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...,CA
4,2Vv-BfVoq4g,17.14.11,Ed Sheeran - Perfect (Official Music Video),Ed Sheeran,10,2017-11-09T11:04:14.000Z,"edsheeran|""ed sheeran""|""acoustic""|""live""|""cove...",33523622,1634130,21082,85067,https://i.ytimg.com/vi/2Vv-BfVoq4g/default.jpg,False,False,False,🎧: https://ad.gt/yt-perfect\n💰: https://atlant...,CA


In [5]:
df_ca.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40881 entries, 0 to 40880
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   video_id                40881 non-null  object
 1   trending_date           40881 non-null  object
 2   title                   40881 non-null  object
 3   channel_title           40881 non-null  object
 4   category_id             40881 non-null  int64 
 5   publish_time            40881 non-null  object
 6   tags                    40881 non-null  object
 7   views                   40881 non-null  int64 
 8   likes                   40881 non-null  int64 
 9   dislikes                40881 non-null  int64 
 10  comment_count           40881 non-null  int64 
 11  thumbnail_link          40881 non-null  object
 12  comments_disabled       40881 non-null  bool  
 13  ratings_disabled        40881 non-null  bool  
 14  video_error_or_removed  40881 non-null  bool  
 15  de

In [6]:
df_ca.describe()

,category_id,views,likes,dislikes,comment_count
count,40881.000000,4.088100e+04,4.088100e+04,4.088100e+04,4.088100e+04
mean,20.795553,1.147036e+06,3.958269e+04,2.009195e+03,5.042975e+03
std,6.775054,3.390913e+06,1.326895e+05,1.900837e+04,2.157902e+04
min,1.000000,7.330000e+02,0.000000e+00,0.000000e+00,0.000000e+00
25%,20.000000,1.439020e+05,2.191000e+03,9.900000e+01,4.170000e+02
50%,24.000000,3.712040e+05,8.780000e+03,3.030000e+02,1.301000e+03
75%,24.000000,9.633020e+05,2.871700e+04,9.500000e+02,3.713000e+03
max,43.000000,1.378431e+08,5.053338e+06,1.602383e+06,1.114800e+06


We made multiple attempts to read and create the Dataframes for each contry, although the Japan one displayed an encoding problem.
We fixed by appliyng utf-8 as encoding format, and replacing faulty bytes. This choice also preserved the original japanese format of the object cells, which, for example, the latin1 encoding(which was suggested by many sources) didnt.

In [7]:
df_jp = pd.read_csv(output_dir + "/JPvideos.csv", encoding = "utf-8", encoding_errors = "replace")
df_jp["country"] = "JP"
df_jp.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,country
0,5ugKfHgsmYw,18.07.02,陸自ヘリ、垂直に落下＝路上の車が撮影,時事通信映像センター,25,2018-02-06T03:04:37.000Z,"事故|""佐賀""|""佐賀県""|""ヘリコプター""|""ヘリ""|""自衛隊""|""墜落""|""落下""|""現...",188085,591,189,0,https://i.ytimg.com/vi/5ugKfHgsmYw/default.jpg,True,False,False,佐賀県神埼市の民家に墜落した陸上自衛隊のＡＨ６４Ｄ戦闘ヘリコプターが垂直に落下する様子を、近...,JP
1,ohObafdd34Y,18.07.02,イッテQ お祭り男宮川×手越 巨大ブランコ②,神谷えりな Kamiya Erina 2,1,2018-02-06T04:01:56.000Z,[none],90929,442,88,174,https://i.ytimg.com/vi/ohObafdd34Y/default.jpg,False,False,False,NaN,JP
2,aBr2kKAHN6M,18.07.02,Live Views of Starman,SpaceX,28,2018-02-06T21:38:22.000Z,[none],6408303,165892,2331,3006,https://i.ytimg.com/vi/aBr2kKAHN6M/default.jpg,False,False,False,NaN,JP
3,5wNnwChvmsQ,18.07.02,東京ディズニーリゾートの元キャストが暴露した秘密5選,アシタノワダイ,25,2018-02-06T06:08:49.000Z,アシタノワダイ,96255,1165,277,545,https://i.ytimg.com/vi/5wNnwChvmsQ/default.jpg,False,False,False,東京ディズニーリゾートの元キャストが暴露した秘密5選\n\nかたまりクリエイトさま\n【検証...,JP
4,B7J47qFvdsk,18.07.02,榮倉奈々、衝撃の死んだふり！映画『家に帰ると妻が必ず死んだふりをしています。』特報,シネマトゥデイ,1,2018-02-06T02:30:00.000Z,[none],108408,1336,74,201,https://i.ytimg.com/vi/B7J47qFvdsk/default.jpg,False,False,False,家に帰ってきたサラリーマンのじゅん（安田顕）は、玄関で血を出して倒れている妻ちえ（榮倉奈々）...,JP


In [8]:
df_jp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20523 entries, 0 to 20522
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   video_id                20523 non-null  object
 1   trending_date           20523 non-null  object
 2   title                   20523 non-null  object
 3   channel_title           20523 non-null  object
 4   category_id             20523 non-null  int64 
 5   publish_time            20523 non-null  object
 6   tags                    20523 non-null  object
 7   views                   20523 non-null  int64 
 8   likes                   20523 non-null  int64 
 9   dislikes                20523 non-null  int64 
 10  comment_count           20523 non-null  int64 
 11  thumbnail_link          20523 non-null  object
 12  comments_disabled       20523 non-null  bool  
 13  ratings_disabled        20523 non-null  bool  
 14  video_error_or_removed  20523 non-null  bool  
 15  de

We had previosly identified line 255 as faulty(video_id 3Xn7nGqiJCU), so we ran a succesful double check in order to see if it was still present.

In [9]:
df_jp.loc[250:260]

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,country
250,DHEgmES0HfU,18.08.02,テラスハウス オープニング ニュー ドアーズ 山チャンネル Vol.7「正直に言います、尊敬...,Netflix Japan,24,2018-02-06T10:00:05.000Z,"ネットフリックス|""ネトフリ""|""半田悠人""|""NETFLIX""|""テラスハウス""|""テラハ...",72459,575,25,250,https://i.ytimg.com/vi/DHEgmES0HfU/default.jpg,False,False,False,【山チャンネル】Netflix公式チャンネル独占公開！山里亮太（南海キャンディーズ）による『...,JP
251,0ovTFlKXcy0,18.08.02,[SUPERSTAR SMTOWN]東方神起プレイ動画,SUPERSTAR SMTOWN,24,2018-02-07T05:23:59.000Z,"東方神起|""SUPERSTAR SMTOWN""|""KPOP""|""音ゲー""|""リズムゲーム""|...",31013,1326,25,63,https://i.ytimg.com/vi/0ovTFlKXcy0/default.jpg,False,False,False,☆東方神起グッズがあたるキャンペーン開催中☆\n\n人気KPOPゲーム【SUPERSTAR ...,JP
252,7UQSgAjKYdk,18.08.02,【7th WEEK】 みずき、元恋人に「早く会いたい…」,TERRACE HOUSE / テラスハウス,24,2018-02-06T09:00:01.000Z,"TERRACE HOUSE|""テラスハウス""|""テラハ""|""映画""|""劇場版""|""NETFL...",114223,332,128,0,https://i.ytimg.com/vi/7UQSgAjKYdk/default.jpg,True,False,False,いよいよテラスハウスの新シーズンがNETFLIXにて配信スタート！ \nその未公開映像をYo...,JP
253,3Xn7nGqiJCU,18.08.02,Why Trump Doesn't Want To Talk To Robert Mueller,The Late Show with Stephen Colbert,24,2018-02-07T08:35:00.000Z,"The Late Show|""Stephen Colbert""|""Colbert""|""Lat...",957641,14094,267,1036,https://i.ytimg.com/vi/3Xn7nGqiJCU/default.jpg,False,False,False,With Trump still undecided about sitting down ...,JP
254,U7pzUOdzYYk,18.08.02,有賀さつき 病気が判明。独りで病魔と戦った決死の覚悟の理由に涙が止まらない…,刺さるニュース,22,2018-02-06T09:00:03.000Z,"刺さるニュース|""UCXYFibd4JtRzi-SsWCvcwww""|""有賀さつき""|""フジ...",223959,321,291,30,https://i.ytimg.com/vi/U7pzUOdzYYk/default.jpg,False,False,False,★チャンネル登録はこちらからお願いいたします★\n⇒https://www.youtube....,JP
255,Fmx-IEa83JU,18.08.02,2018-2-7-01【仮想通貨の投資方法は５つある】視聴者様から家庭用ビデオカメラが届きま...,【コンサルユーチューバー】「法令遵守」「セキュリティ」「ICO資金調達」経営コンサルタント・...,29,2018-02-07T07:49:10.000Z,[none],7757,123,26,24,https://i.ytimg.com/vi/Fmx-IEa83JU/default.jpg,False,False,False,★中野裕二へのご質問や仮想通貨取引所のパソコン設定（スマホ、タブレットも設定します）、人生相...,JP
256,U9Tkv9FjyNw,18.08.02,星野みなみ年表がおもしろすぎるwwww 乃木坂46,最新46と 48,22,2018-02-06T11:56:54.000Z,[none],156835,2555,37,493,https://i.ytimg.com/vi/U9Tkv9FjyNw/default.jpg,False,False,False,NaN,JP
257,NwkoKEOUYaA,18.08.02,猫の噛み癖が治ったと錯覚したあの夜,コロン BB,15,2018-02-07T07:00:01.000Z,"コロンBB|""ライス""|""猫""|""子猫""|""子供""|""面白""|""癒し""|""可愛い""|""ぬこ""...",12442,570,6,103,https://i.ytimg.com/vi/NwkoKEOUYaA/default.jpg,False,False,False,ハメられた,JP
258,T17xaPhX5dU,18.08.02,2/6!!1日千円!!インを頼りに今日も勝つ!!【競艇エブリディ029】,競艇エブリディ,24,2018-02-06T19:03:02.000Z,"競艇|""競艇予想""|""ギャンブル""|""ボートレース""|""競馬""|""競艇エブリディ""|""一日千...",10068,121,13,44,https://i.ytimg.com/vi/T17xaPhX5dU/default.jpg,False,False,False,【競艇エブリディ】\nその名の通り毎日配信予定\n\n少ない投資を心がけてますが\nヒートア...,JP
259,2X6O10gGfzQ,18.08.02,AND MESHのiPhoneX用ケースが新発売！やっと出た人気メッシュデザイン,Appleが大好きなんだよ,26,2018-02-07T08:30:24.000Z,"iPhone X|""AND MESH""|""アンドメッシュ""|""グラデーション""|""iPhon...",7198,112,17,72,https://i.ytimg.com/vi/2X6O10gGfzQ/default.jpg,False,False,False,ずっと今か今かと待っていた方もいらっしゃるのではないでしょうか。\nそのデザインでずっと人気...,JP


While manually checking the dataframe, we noticed that many lines were duplicates! We will have to take this aspect in consideration later.

In [10]:
df_jp[ df_jp['video_id'] == 'DHEgmES0HfU' ]

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,country
162,DHEgmES0HfU,18.08.02,テラスハウス オープニング ニュー ドアーズ 山チャンネル Vol.7「正直に言います、尊敬...,Netflix Japan,24,2018-02-06T10:00:05.000Z,"ネットフリックス|""ネトフリ""|""半田悠人""|""NETFLIX""|""テラスハウス""|""テラハ...",72459,575,25,250,https://i.ytimg.com/vi/DHEgmES0HfU/default.jpg,False,False,False,【山チャンネル】Netflix公式チャンネル独占公開！山里亮太（南海キャンディーズ）による『...,JP
206,DHEgmES0HfU,18.08.02,テラスハウス オープニング ニュー ドアーズ 山チャンネル Vol.7「正直に言います、尊敬...,Netflix Japan,24,2018-02-06T10:00:05.000Z,"ネットフリックス|""ネトフリ""|""半田悠人""|""NETFLIX""|""テラスハウス""|""テラハ...",72459,575,25,250,https://i.ytimg.com/vi/DHEgmES0HfU/default.jpg,False,False,False,【山チャンネル】Netflix公式チャンネル独占公開！山里亮太（南海キャンディーズ）による『...,JP
250,DHEgmES0HfU,18.08.02,テラスハウス オープニング ニュー ドアーズ 山チャンネル Vol.7「正直に言います、尊敬...,Netflix Japan,24,2018-02-06T10:00:05.000Z,"ネットフリックス|""ネトフリ""|""半田悠人""|""NETFLIX""|""テラスハウス""|""テラハ...",72459,575,25,250,https://i.ytimg.com/vi/DHEgmES0HfU/default.jpg,False,False,False,【山チャンネル】Netflix公式チャンネル独占公開！山里亮太（南海キャンディーズ）による『...,JP
294,DHEgmES0HfU,18.08.02,テラスハウス オープニング ニュー ドアーズ 山チャンネル Vol.7「正直に言います、尊敬...,Netflix Japan,24,2018-02-06T10:00:05.000Z,"ネットフリックス|""ネトフリ""|""半田悠人""|""NETFLIX""|""テラスハウス""|""テラハ...",72459,575,25,250,https://i.ytimg.com/vi/DHEgmES0HfU/default.jpg,False,False,False,【山チャンネル】Netflix公式チャンネル独占公開！山里亮太（南海キャンディーズ）による『...,JP


We finally created separate dataFrames for each Country, while adding the Country column

In [11]:
df_de = pd.read_csv(output_dir + "/DEvideos.csv", encoding = "utf-8", encoding_errors = "replace")
df_de["country"] = "DE"

df_fr = pd.read_csv(output_dir + "/FRvideos.csv", encoding = "utf-8", encoding_errors = "replace")
df_fr["country"] = "FR"

df_gb = pd.read_csv(output_dir + "/GBvideos.csv", encoding = "utf-8", encoding_errors = "replace")
df_gb["country"] = "GB"

df_in = pd.read_csv(output_dir + "/INvideos.csv", encoding = "utf-8", encoding_errors = "replace")
df_in["country"] = "IN"

df_jp = pd.read_csv(output_dir + "/JPvideos.csv", encoding = "utf-8", encoding_errors = "replace" )
df_jp["country"] = "JP"

df_kr = pd.read_csv(output_dir + "/KRvideos.csv", encoding = "utf-8", encoding_errors = "replace")
df_kr["country"] = "KR"

df_mx = pd.read_csv(output_dir + "/MXvideos.csv", encoding = "utf-8", encoding_errors = "replace")
df_mx["country"] = "MX"

df_ru = pd.read_csv(output_dir + "/RUvideos.csv", encoding = "utf-8", encoding_errors = "replace")
df_ru["country"] = "RU"

df_us = pd.read_csv(output_dir + "/USvideos.csv", encoding = "utf-8", encoding_errors = "replace")
df_us["country"] = "US"


Here we concatenate all the Data Frames, and move the result to a new, empty one. We had to specify the format of each column of the new dataframes, which otherwise would have been lost.
We used a simple dictionary technique, with the keys as the column names, each paired with their format as value.

In [12]:
from operator import concat


list_df = [df_ca, df_jp, df_de, df_fr, df_gb, df_in, df_kr, df_mx, df_ru, df_us]
col_names = list_df[0].columns
col_types = list_df[0].dtypes.to_dict()
df_result = pd.DataFrame({col: pd.Series(dtype=col_types[col]) for col in col_names})

for df in list_df:
    df_result = pd.concat([df_result, df])

df_result.head()
df_result.info()
df_result.describe()



<class 'pandas.core.frame.DataFrame'>
Index: 375942 entries, 0 to 40948
Data columns (total 17 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   video_id                375942 non-null  object
 1   trending_date           375942 non-null  object
 2   title                   375942 non-null  object
 3   channel_title           375942 non-null  object
 4   category_id             375942 non-null  int64 
 5   publish_time            375942 non-null  object
 6   tags                    375942 non-null  object
 7   views                   375942 non-null  int64 
 8   likes                   375942 non-null  int64 
 9   dislikes                375942 non-null  int64 
 10  comment_count           375942 non-null  int64 
 11  thumbnail_link          375942 non-null  object
 12  comments_disabled       375942 non-null  bool  
 13  ratings_disabled        375942 non-null  bool  
 14  video_error_or_removed  375942 non-null  b

,category_id,views,likes,dislikes,comment_count
count,375942.000000,3.759420e+05,3.759420e+05,3.759420e+05,3.759420e+05
mean,20.232302,1.326568e+06,3.788431e+04,2.126107e+03,4.253775e+03
std,7.132413,7.098568e+06,1.654131e+05,2.248437e+04,2.545876e+04
min,1.000000,1.170000e+02,0.000000e+00,0.000000e+00,0.000000e+00
25%,17.000000,4.697800e+04,6.690000e+02,4.100000e+01,1.090000e+02
50%,23.000000,1.773705e+05,3.446000e+03,1.790000e+02,5.110000e+02
75%,24.000000,6.476792e+05,1.747650e+04,7.490000e+02,2.011000e+03
max,44.000000,4.245389e+08,5.613827e+06,1.944971e+06,1.626501e+06


In [13]:
for df in list_df:
    print(df.shape)


(40881, 17)
(20523, 17)
(40840, 17)
(40724, 17)
(38916, 17)
(37352, 17)
(34567, 17)
(40451, 17)
(40739, 17)
(40949, 17)


In [14]:
total_rows = sum(df.shape[0] for df in list_df)
print(total_rows)


375942


Here we create a new dataframe supposed to contain all videos with no Tag. We tried both by checking for nulls and for entryes with tags shorter than 1 characters. In both cases, the dataframe resulted empty, confirming no missing value.

CORRETTO DOPO AVER COMPLETATO IL PUNTO 8

In [48]:
# df_null_tag = df_result[df_result['tags'].isna()]

# df_null_tag = df_result[df_result['tags'].str.len() <= 1]

df_null_tag = df_result[df_result['tags'] == '[none]']

df_null_tag.info()
df_null_tag.head()


<class 'pandas.core.frame.DataFrame'>
Index: 37698 entries, 41 to 40945
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   video_id                37698 non-null  object
 1   trending_date           37698 non-null  object
 2   title                   37698 non-null  object
 3   channel_title           37698 non-null  object
 4   category_id             37698 non-null  int64 
 5   publish_time            37698 non-null  object
 6   tags                    37698 non-null  object
 7   views                   37698 non-null  int64 
 8   likes                   37698 non-null  int64 
 9   dislikes                37698 non-null  int64 
 10  comment_count           37698 non-null  int64 
 11  thumbnail_link          37698 non-null  object
 12  comments_disabled       37698 non-null  bool  
 13  ratings_disabled        37698 non-null  bool  
 14  video_error_or_removed  37698 non-null  bool  
 15  descri

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,country
41,JwboxqDylgg,17.14.11,Canada Soccer's Women's National Team v USA In...,Canada Soccer,17,2017-11-13T05:53:49.000Z,[none],36311,277,28,13,https://i.ytimg.com/vi/JwboxqDylgg/default.jpg,False,False,False,Canada Soccer's Women's National Team face riv...,CA
58,9B-q8h31Bpk,17.14.11,John Oliver Tackles Louis C.K. And Donald Trum...,TV Shows,22,2017-11-13T04:49:26.000Z,[none],106029,1270,101,181,https://i.ytimg.com/vi/9B-q8h31Bpk/default.jpg,False,False,False,"John Oliver on News, Politics ...",CA
78,1UE5Dq1rvUA,17.14.11,Taylor Swift Perform Ready For It - SNL,Ken Reactz,24,2017-11-12T05:18:02.000Z,[none],320964,8069,285,717,https://i.ytimg.com/vi/1UE5Dq1rvUA/default.jpg,False,False,False,Thanks for watching please subscribe and subsc...,CA
86,pmJQ4KwliX4,17.14.11,"LATEST Q POSTS: ROTHSCHILDS, HOUSE OF SAUD, lL...",James Munder,2,2017-11-12T21:25:40.000Z,[none],116820,1503,139,1066,https://i.ytimg.com/vi/pmJQ4KwliX4/default.jpg,False,False,False,https://pastebin.ca/3930472\n\nSupport My Chan...,CA
98,lHcXhBojpeQ,17.14.11,三屆TVB視帝，拋棄10年青梅竹馬髮妻，為娶小三還不惜與母絕交！,明星百曉生,22,2017-11-12T12:49:50.000Z,[none],88061,47,58,17,https://i.ytimg.com/vi/lHcXhBojpeQ/default.jpg,False,False,False,NaN,CA


Da commentare!

In [16]:
# df_nodup = df_result.drop_duplicates(subset=['video_id', 'trending_date'], keep='first').reset_index(drop=True)
# df_nodup.info()

df_nodup = df_result.drop_duplicates(keep='first').reset_index(drop=True)
df_nodup.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 363372 entries, 0 to 363371
Data columns (total 17 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   video_id                363372 non-null  object
 1   trending_date           363372 non-null  object
 2   title                   363372 non-null  object
 3   channel_title           363372 non-null  object
 4   category_id             363372 non-null  int64 
 5   publish_time            363372 non-null  object
 6   tags                    363372 non-null  object
 7   views                   363372 non-null  int64 
 8   likes                   363372 non-null  int64 
 9   dislikes                363372 non-null  int64 
 10  comment_count           363372 non-null  int64 
 11  thumbnail_link          363372 non-null  object
 12  comments_disabled       363372 non-null  bool  
 13  ratings_disabled        363372 non-null  bool  
 14  video_error_or_removed  363372 non-n

In [17]:
total_views = df_nodup.groupby('channel_title')['views'].sum()
print(total_views)

total_views.to_csv('total_views_per_channel.csv')

channel_title
! 세상에 무슨일이                3942977
!!8時だよ面白ネタ大集合               50207
!BTS・TWICE まとめ               7310
!Los amorosos Virales¡       6069
!t Live                    240038
                           ...   
５分でできる DIY                 224232
ＢＡＳＨｔｖ                     116292
Ｋのフィッシングちゃんねる               37165
ＷＡＪＯＩＫＡ                    424129
💜Sandrea                   347155
Name: views, Length: 37824, dtype: int64


Clean solution of point X. No particular problems were encountered at this point.

In [18]:
df_excluded = df_nodup[((df_nodup['comments_disabled'] == True) & (df_nodup['ratings_disabled'] == True)) | (df_nodup['video_error_or_removed'] == True)]
df_excluded.info()
df_excluded.head()

df_excluded.to_excel('excluded_videos.xlsx', index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 2438 entries, 173 to 357065
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   video_id                2438 non-null   object
 1   trending_date           2438 non-null   object
 2   title                   2438 non-null   object
 3   channel_title           2438 non-null   object
 4   category_id             2438 non-null   int64 
 5   publish_time            2438 non-null   object
 6   tags                    2438 non-null   object
 7   views                   2438 non-null   int64 
 8   likes                   2438 non-null   int64 
 9   dislikes                2438 non-null   int64 
 10  comment_count           2438 non-null   int64 
 11  thumbnail_link          2438 non-null   object
 12  comments_disabled       2438 non-null   bool  
 13  ratings_disabled        2438 non-null   bool  
 14  video_error_or_removed  2438 non-null   bool  
 15  descr

In [19]:
df_nodup[['comments_disabled', 'ratings_disabled', 'video_error_or_removed']].value_counts()

comments_disabled  ratings_disabled  video_error_or_removed
False              False             False                     350644
True               False             False                       5713
False              True              False                       4577
True               True              False                       2194
False              False             True                         244
Name: count, dtype: int64

In [20]:
df_no_excluded = df_nodup[~(((df_nodup['comments_disabled'] == True) & (df_nodup['ratings_disabled'] == True)) | (df_nodup['video_error_or_removed'] == True))]

df_no_excluded.info()
df_no_excluded.head()


<class 'pandas.core.frame.DataFrame'>
Index: 360934 entries, 0 to 363371
Data columns (total 17 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   video_id                360934 non-null  object
 1   trending_date           360934 non-null  object
 2   title                   360934 non-null  object
 3   channel_title           360934 non-null  object
 4   category_id             360934 non-null  int64 
 5   publish_time            360934 non-null  object
 6   tags                    360934 non-null  object
 7   views                   360934 non-null  int64 
 8   likes                   360934 non-null  int64 
 9   dislikes                360934 non-null  int64 
 10  comment_count           360934 non-null  int64 
 11  thumbnail_link          360934 non-null  object
 12  comments_disabled       360934 non-null  bool  
 13  ratings_disabled        360934 non-null  bool  
 14  video_error_or_removed  360934 non-null  

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,country
0,n1WpP7iowLc,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,2017-11-10T17:00:03.000Z,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787425,43420,125882,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. Beyoncé i...,CA
1,0dBIkQ4Mz1M,17.14.11,PLUSH - Bad Unboxing Fan Mail,iDubbbzTV,23,2017-11-13T17:00:00.000Z,"plush|""bad unboxing""|""unboxing""|""fan mail""|""id...",1014651,127794,1688,13030,https://i.ytimg.com/vi/0dBIkQ4Mz1M/default.jpg,False,False,False,STill got a lot of packages. Probably will las...,CA
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146035,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...,CA
3,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095828,132239,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...,CA
4,2Vv-BfVoq4g,17.14.11,Ed Sheeran - Perfect (Official Music Video),Ed Sheeran,10,2017-11-09T11:04:14.000Z,"edsheeran|""ed sheeran""|""acoustic""|""live""|""cove...",33523622,1634130,21082,85067,https://i.ytimg.com/vi/2Vv-BfVoq4g/default.jpg,False,False,False,🎧: https://ad.gt/yt-perfect\n💰: https://atlant...,CA


In [21]:
df_no_excluded[['comments_disabled', 'ratings_disabled', 'video_error_or_removed']].value_counts()

comments_disabled  ratings_disabled  video_error_or_removed
False              False             False                     350644
True               False             False                       5713
False              True              False                       4577
Name: count, dtype: int64

In [22]:
df_no_excluded[df_no_excluded['dislikes'] == 0]
# it is strange that there are videos with 0 dislikes in trending youtube videos, probably botting problems

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,country
67,amEZKmJQ4Io,17.14.11,Drako - Watch Me Do It [Official Video],babygranderecords,10,2017-10-23T19:38:36.000Z,"Drako|""Watch Me Do It""|""watch me""|""migos""|""dap...",25887,0,0,6,https://i.ytimg.com/vi/amEZKmJQ4Io/default.jpg,False,True,False,PURCHASE / STREAM WATCH ME DO IT https://fanli...,CA
161,bsgocWaULIU,17.14.11,Troydon bent - Jam African,Troydon bent,10,2016-12-08T19:29:25.000Z,New Danchall music world reggae traditional so...,16921,187,0,5,https://i.ytimg.com/vi/bsgocWaULIU/default.jpg,False,False,False,Troydon bent performing Jam African produced b...,CA
235,wDEA3rpYHnI,17.15.11,Marie-Louise Arsenault réplique à Denise Bomba...,TV Classics,22,2017-11-13T01:26:37.000Z,Marie-Louise Arsenault qui réplique à Denise B...,15800,88,0,0,https://i.ytimg.com/vi/wDEA3rpYHnI/default.jpg,True,False,False,Moment favori à la télé québécoise: Marie-Loui...,CA
271,LfajgipdxmA,17.15.11,How does Yoonla work? Learning CPA,Anthonys Digital Lifestyle Dotcom,22,2017-11-14T10:57:38.000Z,"How does Yoonla work|""free work from home""|""le...",11810,1,0,0,https://i.ytimg.com/vi/LfajgipdxmA/default.jpg,False,False,False,How does Yoonla work? Learning CPA marketing t...,CA
313,8sdQ7dP9Drs,17.15.11,"Today's News: It's On - Get The Goods, Novembe...",Whistler Blackcomb,17,2017-11-14T18:18:21.000Z,[none],4851,31,0,0,https://i.ytimg.com/vi/8sdQ7dP9Drs/default.jpg,False,False,False,Today’s news: it’s on. Whistler Mountain will ...,CA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345917,8WbxbqwdWog,18.13.03,Camera Goes on Japanese Sushi Conveyor Belt Sh...,TkyoSam,22,2018-03-06T16:18:30.000Z,"東京サム|""jvlog""|""jvlogger""|""tkyosam""|""tokyo""|""sam...",639517,0,0,1059,https://i.ytimg.com/vi/8WbxbqwdWog/default.jpg,False,True,False,I will call this video A Camera's Journey Thro...,US
348022,zkrq7Kpd1so,18.24.03,ULTRA LIVE presents Ultra Music Festival 2018 ...,UMF TV,10,2018-03-24T07:55:21.000Z,"ultra music festival|""umf""|""mix""|""relive""|""aft...",1648886,0,0,548,https://i.ytimg.com/vi/zkrq7Kpd1so/default.jpg,False,True,False,Tune in here for the Ultra Music Festival 2018...,US
351021,9TUBf6l7FBg,18.14.04,Coachella 2018 LIVE Channel 1,Coachella,10,2018-04-05T06:48:28.000Z,[none],3598220,0,0,0,https://i.ytimg.com/vi/9TUBf6l7FBg/default_liv...,False,True,False,For more cameras and VR180 immersive experienc...,US
351222,9TUBf6l7FBg,18.15.04,Coachella 2018 LIVE Channel 1,Coachella,10,2018-04-05T06:48:28.000Z,[none],11137071,0,0,2,https://i.ytimg.com/vi/9TUBf6l7FBg/default_liv...,False,True,False,For more cameras and VR180 immersive experienc...,US


In [23]:
# for i in range(len(df_no_excluded)):
#     if df_no_excluded['dislikes'] == 0:
#         df_no_excluded['like_ratio'] = df_no_excluded['likes']
#     else:
#         df_no_excluded['like_ratio'] = df_no_excluded['likes'] / df_no_excluded['dislikes']

# df_no_excluded.head()


df_no_excluded['like_ratio'] = np.where(
    df_no_excluded['dislikes'] == 0,
    df_no_excluded['likes'], 
    df_no_excluded['likes'] / df_no_excluded['dislikes']
)
df_no_excluded.head()

C:\Users\artus\AppData\Local\Temp\ipykernel_23040\4057421799.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_excluded['like_ratio'] = np.where(


,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,country,like_ratio
0,n1WpP7iowLc,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,2017-11-10T17:00:03.000Z,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787425,43420,125882,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. Beyoncé i...,CA,18.135076
1,0dBIkQ4Mz1M,17.14.11,PLUSH - Bad Unboxing Fan Mail,iDubbbzTV,23,2017-11-13T17:00:00.000Z,"plush|""bad unboxing""|""unboxing""|""fan mail""|""id...",1014651,127794,1688,13030,https://i.ytimg.com/vi/0dBIkQ4Mz1M/default.jpg,False,False,False,STill got a lot of packages. Probably will las...,CA,75.707346
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146035,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...,CA,27.352500
3,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095828,132239,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...,CA,66.485168
4,2Vv-BfVoq4g,17.14.11,Ed Sheeran - Perfect (Official Music Video),Ed Sheeran,10,2017-11-09T11:04:14.000Z,"edsheeran|""ed sheeran""|""acoustic""|""live""|""cove...",33523622,1634130,21082,85067,https://i.ytimg.com/vi/2Vv-BfVoq4g/default.jpg,False,False,False,🎧: https://ad.gt/yt-perfect\n💰: https://atlant...,CA,77.513044


In [24]:
df_no_excluded[df_no_excluded['dislikes'] == 0].head(5)

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,country,like_ratio
67,amEZKmJQ4Io,17.14.11,Drako - Watch Me Do It [Official Video],babygranderecords,10,2017-10-23T19:38:36.000Z,"Drako|""Watch Me Do It""|""watch me""|""migos""|""dap...",25887,0,0,6,https://i.ytimg.com/vi/amEZKmJQ4Io/default.jpg,False,True,False,PURCHASE / STREAM WATCH ME DO IT https://fanli...,CA,0.0
161,bsgocWaULIU,17.14.11,Troydon bent - Jam African,Troydon bent,10,2016-12-08T19:29:25.000Z,New Danchall music world reggae traditional so...,16921,187,0,5,https://i.ytimg.com/vi/bsgocWaULIU/default.jpg,False,False,False,Troydon bent performing Jam African produced b...,CA,187.0
235,wDEA3rpYHnI,17.15.11,Marie-Louise Arsenault réplique à Denise Bomba...,TV Classics,22,2017-11-13T01:26:37.000Z,Marie-Louise Arsenault qui réplique à Denise B...,15800,88,0,0,https://i.ytimg.com/vi/wDEA3rpYHnI/default.jpg,True,False,False,Moment favori à la télé québécoise: Marie-Loui...,CA,88.0
271,LfajgipdxmA,17.15.11,How does Yoonla work? Learning CPA,Anthonys Digital Lifestyle Dotcom,22,2017-11-14T10:57:38.000Z,"How does Yoonla work|""free work from home""|""le...",11810,1,0,0,https://i.ytimg.com/vi/LfajgipdxmA/default.jpg,False,False,False,How does Yoonla work? Learning CPA marketing t...,CA,1.0
313,8sdQ7dP9Drs,17.15.11,"Today's News: It's On - Get The Goods, Novembe...",Whistler Blackcomb,17,2017-11-14T18:18:21.000Z,[none],4851,31,0,0,https://i.ytimg.com/vi/8sdQ7dP9Drs/default.jpg,False,False,False,Today’s news: it’s on. Whistler Mountain will ...,CA,31.0


In [25]:
df_nodup['publish_time'] = pd.to_datetime(df_nodup['publish_time'])

In [26]:
df_nodup.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,country
0,n1WpP7iowLc,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,2017-11-10 17:00:03+00:00,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787425,43420,125882,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. Beyoncé i...,CA
1,0dBIkQ4Mz1M,17.14.11,PLUSH - Bad Unboxing Fan Mail,iDubbbzTV,23,2017-11-13 17:00:00+00:00,"plush|""bad unboxing""|""unboxing""|""fan mail""|""id...",1014651,127794,1688,13030,https://i.ytimg.com/vi/0dBIkQ4Mz1M/default.jpg,False,False,False,STill got a lot of packages. Probably will las...,CA
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12 19:05:24+00:00,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146035,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...,CA
3,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12 18:01:41+00:00,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095828,132239,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...,CA
4,2Vv-BfVoq4g,17.14.11,Ed Sheeran - Perfect (Official Music Video),Ed Sheeran,10,2017-11-09 11:04:14+00:00,"edsheeran|""ed sheeran""|""acoustic""|""live""|""cove...",33523622,1634130,21082,85067,https://i.ytimg.com/vi/2Vv-BfVoq4g/default.jpg,False,False,False,🎧: https://ad.gt/yt-perfect\n💰: https://atlant...,CA


In [27]:
df_nodup['publish_min'] = df_nodup['publish_time'].dt.strftime('%M')

df_nodup['publish_min_unit'] = df_nodup['publish_min'].astype(int) % 10

df_nodup['inferior_lim'] = df_nodup['publish_min'].astype(int) - df_nodup['publish_min_unit']

df_nodup['superior_lim'] = df_nodup['inferior_lim'] + 10

df_nodup['raw_cluster'] = "from " + df_nodup['publish_time'].dt.strftime('%H') + ":" + df_nodup['inferior_lim'].astype(str).str.zfill(2) + " to " + df_nodup['publish_time'].dt.strftime('%H') + ":" + df_nodup['superior_lim'].astype(str).str.zfill(2)

df_nodup['cluster'] = np.where(
    df_nodup['superior_lim'].astype(int) == 60,
    "from " + df_nodup['publish_time'].dt.strftime('%H') + ":" + df_nodup['inferior_lim'].astype(str).str.zfill(2) + " to " + (df_nodup['publish_time'] + pd.Timedelta(hours=1)).dt.strftime('%H') + ":00", 
    "from " + df_nodup['publish_time'].dt.strftime('%H') + ":" + df_nodup['inferior_lim'].astype(str).str.zfill(2) + " to " + df_nodup['publish_time'].dt.strftime('%H') + ":" + df_nodup['superior_lim'].astype(str).str.zfill(2)
)

df_nodup.drop(columns=['publish_min', 'inferior_lim', 'superior_lim', 'publish_min_unit', 'raw_cluster'], inplace=True)

df_nodup.head(20)



,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,country,cluster
0,n1WpP7iowLc,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,2017-11-10 17:00:03+00:00,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787425,43420,125882,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. Beyoncé i...,CA,from 17:00 to 17:10
1,0dBIkQ4Mz1M,17.14.11,PLUSH - Bad Unboxing Fan Mail,iDubbbzTV,23,2017-11-13 17:00:00+00:00,"plush|""bad unboxing""|""unboxing""|""fan mail""|""id...",1014651,127794,1688,13030,https://i.ytimg.com/vi/0dBIkQ4Mz1M/default.jpg,False,False,False,STill got a lot of packages. Probably will las...,CA,from 17:00 to 17:10
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12 19:05:24+00:00,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146035,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...,CA,from 19:00 to 19:10
3,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12 18:01:41+00:00,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095828,132239,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...,CA,from 18:00 to 18:10
4,2Vv-BfVoq4g,17.14.11,Ed Sheeran - Perfect (Official Music Video),Ed Sheeran,10,2017-11-09 11:04:14+00:00,"edsheeran|""ed sheeran""|""acoustic""|""live""|""cove...",33523622,1634130,21082,85067,https://i.ytimg.com/vi/2Vv-BfVoq4g/default.jpg,False,False,False,🎧: https://ad.gt/yt-perfect\n💰: https://atlant...,CA,from 11:00 to 11:10
5,0yIWz1XEeyc,17.14.11,Jake Paul Says Alissa Violet CHEATED with LOGA...,DramaAlert,25,2017-11-13 07:37:51+00:00,"#DramaAlert|""Drama""|""Alert""|""DramaAlert""|""keem...",1309699,103755,4613,12143,https://i.ytimg.com/vi/0yIWz1XEeyc/default.jpg,False,False,False,► Follow for News! - https://twitter.com/KEEMS...,CA,from 07:30 to 07:40
6,_uM5kFfkhB8,17.14.11,Vanoss Superhero School - New Students,VanossGaming,23,2017-11-12 23:52:13+00:00,"Funny Moments|""Montage video games""|""gaming""|""...",2987945,187464,9850,26629,https://i.ytimg.com/vi/_uM5kFfkhB8/default.jpg,False,False,False,Vanoss Merch Shop: https://vanoss.3blackdot.co...,CA,from 23:50 to 00:00
7,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13 17:13:01+00:00,SHANtell martin,748374,57534,2967,15959,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...,CA,from 17:10 to 17:20
8,JzCsM1vtn78,17.14.11,THE LOGANG MADE HISTORY. LOL. AGAIN.,Logan Paul Vlogs,24,2017-11-12 20:19:24+00:00,"logan paul vlog|""logan paul""|""logan""|""paul""|""o...",4477587,292837,4123,36391,https://i.ytimg.com/vi/JzCsM1vtn78/default.jpg,False,False,False,Join the movement. Be a Maverick ► https://Sho...,CA,from 20:10 to 20:20
9,43sm-QwLcx4,17.14.11,Finally Sheldon is winning an argument about t...,Sheikh Musa,22,2017-11-10 14:10:46+00:00,"God|""Sheldon Cooper""|""Young Sheldon""",505161,4135,976,1484,https://i.ytimg.com/vi/43sm-QwLcx4/default.jpg,False,False,False,Sheldon is roasting pastor of the church\nyoun...,CA,from 14:10 to 14:20


In [28]:
cluster_metrics = df_nodup.groupby('cluster').agg(
    distinct_videos=('video_id', 'nunique'),
    avg_likes=('likes', 'mean'),
    avg_dislikes=('dislikes', 'mean')
)

cluster_metrics.head(144)




,distinct_videos,avg_likes,avg_dislikes
cluster,,,
from 00:00 to 00:10,1381,62733.717976,3895.373319
from 00:10 to 00:20,750,22889.228224,1469.852802
from 00:20 to 00:30,653,21692.678396,1087.520458
from 00:30 to 00:40,841,37906.337812,970.609085
from 00:40 to 00:50,642,43186.191129,1951.069355
...,...,...,...
from 23:10 to 23:20,812,23098.396461,1125.621887
from 23:20 to 23:30,732,30439.792334,1170.986760
from 23:30 to 23:40,915,18345.056707,881.782927


Point N8

In [68]:
df_result['trending_date'] = pd.to_datetime(df_result['trending_date'], format='%y.%d.%m')
df_result_ordered = df_result.sort_values(by=['trending_date'], ascending=[True])
df_no_video_dup = df_result_ordered.drop_duplicates(subset=['video_id'], keep='last').reset_index(drop=True)

In [70]:

df_no_video_dup.info()
df_no_video_dup.tail()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184287 entries, 0 to 184286
Data columns (total 17 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   video_id                184287 non-null  object        
 1   trending_date           184287 non-null  datetime64[ns]
 2   title                   184287 non-null  object        
 3   channel_title           184287 non-null  object        
 4   category_id             184287 non-null  int64         
 5   publish_time            184287 non-null  object        
 6   tags                    184287 non-null  object        
 7   views                   184287 non-null  int64         
 8   likes                   184287 non-null  int64         
 9   dislikes                184287 non-null  int64         
 10  comment_count           184287 non-null  int64         
 11  thumbnail_link          184287 non-null  object        
 12  comments_disabled       184287

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,country
184282,yerrtSj1fF4,2018-06-14,Le tardigrade est l'animal le plus indestructi...,Brut,25,2018-06-13T08:15:00.000Z,"tardigrade|""ourson d'eau""|""animaux""",14324,561,7,137,https://i.ytimg.com/vi/yerrtSj1fF4/default.jpg,False,False,False,"Quasi-indestructibles, ils ne craignent ni les...",FR
184283,zTCtqR8cbuA,2018-06-14,Manon Marsault découvrez son accouchement en D...,Snap Stars,25,2018-06-12T16:56:01.000Z,Manon marsault julien tanti Manon et JULIEN LE...,15048,224,6,17,https://i.ytimg.com/vi/zTCtqR8cbuA/default.jpg,False,False,False,NaN,FR
184284,6Bp_hw0VrEw,2018-06-14,LA REVANCHE DE PAUL KABESA !,Boumé-Sama,24,2018-06-13T11:09:58.000Z,"paul kabesa|""DTT""|""jay max""|""boumé""|""boume""|""b...",23609,1835,13,86,https://i.ytimg.com/vi/6Bp_hw0VrEw/default.jpg,False,False,False,On continu la série coupe du monde 2 vidéos pa...,FR
184285,QUNUvGESI_U,2018-06-14,MEDINE au Bataclan : retour sur la polémique -...,OKLMofficial,24,2018-06-12T14:15:14.000Z,"Mehdi Maizi|""La Sauce""|""OKLMofficial""|""OKLM Ra...",16476,539,41,149,https://i.ytimg.com/vi/QUNUvGESI_U/default.jpg,False,False,False,Abonne toi: http://bit.ly/2ke5AOiMédine Au Bat...,FR
184286,ooyjaVdt-jA,2018-06-14,Official Call of Duty®: Black Ops 4 — Multipla...,Call of Duty,20,2018-05-17T17:09:38.000Z,"call of duty|""cod""|""activision""|""Black Ops 4""",10306119,357079,212976,144795,https://i.ytimg.com/vi/ooyjaVdt-jA/default.jpg,False,False,False,Call of Duty: Black Ops 4 Multiplayer raises t...,US


In [40]:
tags_list = []
for tags in df_no_video_dup['tags']:
    tags_split = tags.split('|')
    for tag in tags_split:
        tag = tag.strip('"').strip("'")
        tags_list.append(tag)

In [42]:
print(tags_list[:20])

['Eminem', 'Walk', 'On', 'Water', 'Aftermath/Shady/Interscope', 'Rap', 'plush', 'bad unboxing', 'unboxing', 'fan mail', 'idubbbztv', 'idubbbztv2', 'things', 'best', 'packages', 'plushies', 'chontent chop', 'racist superman', 'rudy', 'mancuso']


In [ ]:

tags_apperances = Counter(tags_list)
df_tags_apperances = pd.DataFrame.from_dict(tags_apperances, orient='index', columns=['appearances'])

In [53]:
tags_apperances

Counter({'[none]': 23971,
         '2018': 5756,
         'funny': 4136,
         'comedy': 3101,
         'news': 3082,
         '2017': 2655,
         'video': 2075,
         'show': 2035,
         'tv': 1730,
         'television': 1720,
         'новости': 1551,
         'music': 1512,
         'humour': 1423,
         'политика': 1423,
         'юмор': 1410,
         'vlog': 1407,
         'humor': 1334,
         'TV': 1325,
         'обзор': 1312,
         'youtube': 1305,
         'review': 1303,
         'live': 1285,
         'football': 1270,
         'diy': 1254,
         'путин': 1224,
         'serial': 1212,
         'interview': 1153,
         'challenge': 1127,
         'News': 1123,
         'rap': 1122,
         'food': 1100,
         'politics': 1056,
         'DIY': 1052,
         'noticias': 1033,
         'top 10': 1023,
         'full episode': 999,
         'Россия': 993,
         'mexico': 988,
         'top': 985,
         'drama': 968,
         '먹방': 963,
   

In [59]:

df_tags_apperances = df_tags_apperances.sort_values(by='appearances', ascending=False)
print(df_tags_apperances[1:6])

        appearances
2018           5756
funny          4136
comedy         3101
news           3082
2017           2655


In [ ]:
df_no_video_dup['tag_list'] = df_no_video_dup['tags'].apply(lambda x: [tag.strip('"').strip("'") for tag in x.split('|')])



In [72]:
df_no_video_dup.info()
df_no_video_dup.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184287 entries, 0 to 184286
Data columns (total 18 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   video_id                184287 non-null  object        
 1   trending_date           184287 non-null  datetime64[ns]
 2   title                   184287 non-null  object        
 3   channel_title           184287 non-null  object        
 4   category_id             184287 non-null  int64         
 5   publish_time            184287 non-null  object        
 6   tags                    184287 non-null  object        
 7   views                   184287 non-null  int64         
 8   likes                   184287 non-null  int64         
 9   dislikes                184287 non-null  int64         
 10  comment_count           184287 non-null  int64         
 11  thumbnail_link          184287 non-null  object        
 12  comments_disabled       184287

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,country,tag_list
0,y5wvD7QH1x4,2017-11-14,20 Chistes malos que te harán reír #12,ElJamesHD Tops,24,2017-11-12T19:00:04.000Z,"20 Chistes malos que te harán reír|""chistes ma...",299832,44011,1213,14355,https://i.ytimg.com/vi/y5wvD7QH1x4/default.jpg,False,False,False,Descarga la app: http://bit.do/ElJamesHDTops_Y...,MX,"[20 Chistes malos que te harán reír, chistes m..."
1,IZRJsmfscIg,2017-11-14,Lomo En Salsa de mostaza sin Horno,Armando En Tu Cocina,26,2017-11-13T17:01:14.000Z,"lomo de cerdo en salsa de moztaza|""armando en ...",14409,2419,17,441,https://i.ytimg.com/vi/IZRJsmfscIg/default.jpg,False,False,False,Les dejo este delicioso lomo de cerdo en salsa...,MX,"[lomo de cerdo en salsa de moztaza, armando en..."
2,URnaV-AK4og,2017-11-14,Le Lloran al INE Margarita Zavala y El Bronco ...,Campechaneando,25,2017-11-13T01:36:29.000Z,"peña nieto|""epn""|""amlo""|""lopez obrador""|""marga...",31974,2663,50,500,https://i.ytimg.com/vi/URnaV-AK4og/default.jpg,False,False,False,Los independientes piden más tiempo al INE ¡Po...,MX,"[peña nieto, epn, amlo, lopez obrador, margari..."
3,s-VstUmpPzM,2017-11-14,TERREMOTO 7.6 EN IRAK 2017 ALERTA MUNDIAL,ALERTA MUNDIAL,22,2017-11-12T20:35:38.000Z,[none],34707,97,28,24,https://i.ytimg.com/vi/s-VstUmpPzM/default.jpg,False,False,False,NaN,MX,[[none]]
4,OIBG99riACs,2017-11-14,TEXTO DIARIO ANIMADO 13 de noviembre 2017,TextoDiarioAnimado,27,2017-11-13T03:57:01.000Z,"texto diario|""animado""|""jw.org""",8090,262,3,4,https://i.ytimg.com/vi/OIBG99riACs/default.jpg,False,False,False,Lunes 13 de noviembre. Verán al Hijo del hombr...,MX,"[texto diario, animado, jw.org]"


In [ ]:
df_exploded_tags = df_no_video_dup.explode('tag_list').reset_index(drop=True)
# df_exploded_tags = df_exploded_tags[df_exploded_tags['tag_list'] != '[none]']
df_exploded_tags.info()
df_exploded_tags.head()


<class 'pandas.core.frame.DataFrame'>
Index: 2973850 entries, 0 to 2997743
Data columns (total 18 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   video_id                object        
 1   trending_date           datetime64[ns]
 2   title                   object        
 3   channel_title           object        
 4   category_id             int64         
 5   publish_time            object        
 6   tags                    object        
 7   views                   int64         
 8   likes                   int64         
 9   dislikes                int64         
 10  comment_count           int64         
 11  thumbnail_link          object        
 12  comments_disabled       bool          
 13  ratings_disabled        bool          
 14  video_error_or_removed  bool          
 15  description             object        
 16  country                 object        
 17  tag_list                object        
dtypes: bool

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,country,tag_list
0,y5wvD7QH1x4,2017-11-14,20 Chistes malos que te harán reír #12,ElJamesHD Tops,24,2017-11-12T19:00:04.000Z,"20 Chistes malos que te harán reír|""chistes ma...",299832,44011,1213,14355,https://i.ytimg.com/vi/y5wvD7QH1x4/default.jpg,False,False,False,Descarga la app: http://bit.do/ElJamesHDTops_Y...,MX,20 Chistes malos que te harán reír
1,y5wvD7QH1x4,2017-11-14,20 Chistes malos que te harán reír #12,ElJamesHD Tops,24,2017-11-12T19:00:04.000Z,"20 Chistes malos que te harán reír|""chistes ma...",299832,44011,1213,14355,https://i.ytimg.com/vi/y5wvD7QH1x4/default.jpg,False,False,False,Descarga la app: http://bit.do/ElJamesHDTops_Y...,MX,chistes malos
2,y5wvD7QH1x4,2017-11-14,20 Chistes malos que te harán reír #12,ElJamesHD Tops,24,2017-11-12T19:00:04.000Z,"20 Chistes malos que te harán reír|""chistes ma...",299832,44011,1213,14355,https://i.ytimg.com/vi/y5wvD7QH1x4/default.jpg,False,False,False,Descarga la app: http://bit.do/ElJamesHDTops_Y...,MX,chistes
3,y5wvD7QH1x4,2017-11-14,20 Chistes malos que te harán reír #12,ElJamesHD Tops,24,2017-11-12T19:00:04.000Z,"20 Chistes malos que te harán reír|""chistes ma...",299832,44011,1213,14355,https://i.ytimg.com/vi/y5wvD7QH1x4/default.jpg,False,False,False,Descarga la app: http://bit.do/ElJamesHDTops_Y...,MX,chistes graciosos
4,y5wvD7QH1x4,2017-11-14,20 Chistes malos que te harán reír #12,ElJamesHD Tops,24,2017-11-12T19:00:04.000Z,"20 Chistes malos que te harán reír|""chistes ma...",299832,44011,1213,14355,https://i.ytimg.com/vi/y5wvD7QH1x4/default.jpg,False,False,False,Descarga la app: http://bit.do/ElJamesHDTops_Y...,MX,mejores chistes


In [80]:
df_tags_country_ratio = df_exploded_tags.groupby(['tag_list', 'country']).agg(
    sum_likes = ('likes', 'sum'),
    sum_dislikes = ('dislikes', 'sum')
)

In [81]:
df_tags_country_ratio.head(20)

sum_likes  \
tag_list                                           country              
                                                   CA          148686   
                                                   DE           14160   
                                                   FR           11017   
                                                   IN          634512   
                                                   KR            6951   
                                                   MX           49268   
                                                   RU           13020   
                                                   US          160720   
 #Freeticket                                       IN            3085   
 #GST                                              IN            2632   
 #Jaisimha                                         IN            3085   
 #JanaSenaParty                                    IN            4729   
 #MahaaNews                                        IN            2048   
 #PawanKalyan #AlluArjun                           IN            2373   
 #PrimeTimeWithMurthy                              IN            9589   
 #RamGopalVarma                                    IN            2632   
 #SUSANNE                                          DE            6465   
 #VMA 20                                           FR             875   
 #VMA 21                                           FR             717   
 ''A por la FORTALEZA'' Capitulo 88 - Segunda P... MX             727   

                                                            sum_dislikes  
tag_list                                           country                
                                                   CA              18637  
                                                   DE               1096  
                                                   FR                557  
                                                   IN             104422  
                                                   KR               6427  
                                                   MX               5687  
                                                   RU                790  
                                                   US               9835  
 #Freeticket                                       IN               1162  
 #GST                                              IN               1089  
 #Jaisimha                                         IN               1162  
 #JanaSenaParty                                    IN                276  
 #MahaaNews                                        IN                246  
 #PawanKalyan #AlluArjun                           IN                252  
 #PrimeTimeWithMurthy                              IN               1978  
 #RamGopalVarma                                    IN               1089  
 #SUSANNE                                          DE                 82  
 #VMA 20                                           FR                 91  
 #VMA 21                                           FR                124  
 ''A por la FORTALEZA'' Capitulo 88 - Segunda P... MX                270